# 서울스토어 크롤러

서울스토어 Women's 페이지의 상품(이미지, 상세정보 링크)을 크롤링하는 크롤러입니다.

크롤링 결과는 {"key": 상품url, "img_url": 이미지url} 형태의 json 파일로 저장합니다.

In [2]:
from __future__ import unicode_literals
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import glob
import json
import os
import re
import sys
import time
import traceback
from builtins import open
from time import sleep

In [82]:
root = 'https://www.seoulstore.com/compilations/582/products'
key_set = set()
posts = []

def get_products(num, wait_time):
    browser = webdriver.Chrome('chromedriver')
    wait = WebDriverWait(browser, wait_time)
    browser.get(root)
    body = browser.find_element_by_tag_name('body')
    
    count = 0
    prev_posts_count = 0
    
    while len(posts) < num:
        
        body.send_keys(Keys.PAGE_DOWN)
        ele_posts = browser.find_elements_by_class_name('image_container')
        cur_posts_count = len(ele_posts)

        if prev_posts_count == cur_posts_count:
            count += 1
        
        else:   #로드된 데이터가 더 있다면 count를 0으로 초기화
            count = 0
            
        if count == 50:  #스크롤을 50회 이상했는데도 추가로 로드되는 데이터가 더 이상 없을 시 break
            break
            
        for ele in ele_posts:
            key = ele.find_element_by_tag_name('a').get_attribute('href')
            if key not in key_set:
                dict_post = { "key": key }
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
                ele_img = ele.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
                dict_post["img_url"] = ele_img.get_attribute("src")
                key_set.add(key)
                posts.append(dict_post)
                
        prev_posts_count = cur_posts_count
        
    return posts[:num]

def output(data, filepath):
    out = json.dumps(data, ensure_ascii=False)
    if filepath:
        with open(filepath, "w", encoding="utf8") as f:
            f.write(out)
    else:
        print(out)

In [83]:
output(get_products(1000, 300), '../output_seoulstore.json')

In [84]:
with open('../output_seoulstore.json') as data_file:
    data = json.load(data_file)

In [85]:
len(data)

381